In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credential/YT Credential.txt
/kaggle/input/credential/Reddit credential.txt


In [2]:
#!pip3 install google-api-python-client

In [3]:
import googleapiclient.discovery
from IPython.display import JSON

In [4]:
with open("/kaggle/input/credential/YT Credential.txt", "r") as f:
    file = f.read()
file = file.split()
key = file[0]

In [5]:
# Disable OAuthlib's HTTPS verification when running locally.
# *DO NOT* leave this option enabled in production.

os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = key

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey = DEVELOPER_KEY)

In [6]:
def YTcomments(videoId):
    all_data = []
    itemCount = 0
    
    #First page
    request = youtube.commentThreads().list(part="id, replies, snippet", videoId=videoId, maxResults="100", textFormat="html")
    response = request.execute()
    next_page_token = response.get('nextPageToken')
    
    for item in response["items"]:
        #print(item["replies"])

        if "replies" not in item:
            data = {"Thread": itemCount,
                    "Comment": item["snippet"]["topLevelComment"]["snippet"]["textOriginal"],
                    "Replies": 0,
                    "Likes": item["snippet"]["topLevelComment"]["snippet"]["likeCount"]}
            all_data.append(data)
        else:
            data = {"Thread": itemCount,
                    "Comment": item["snippet"]["topLevelComment"]["snippet"]["textOriginal"],
                    "Replies": 1,
                    "Likes": item["snippet"]["topLevelComment"]["snippet"]["likeCount"]}
            all_data.append(data)

            replyCount = 0
            for reply in item["replies"]["comments"]:
                data = {"Thread": f"{itemCount}.{replyCount}",
                        "Comment": reply["snippet"]["textOriginal"],
                        "Replies": 2,
                        "Likes":reply["snippet"]["likeCount"]}
                all_data.append(data)
                replyCount += 1
        itemCount += 1
    
    ########################################################################################################
    #Other pages
    while next_page_token is not None:
        #print(next_page_token)
        request = youtube.commentThreads().list(part="id, replies, snippet", videoId=videoId, maxResults="100", pageToken=next_page_token)
        response = request.execute()
        
        for item in response["items"]:
            #print(item["replies"])
            
            if "replies" not in item:
                data = {"Thread": itemCount,
                        "Comment": item["snippet"]["topLevelComment"]["snippet"]["textOriginal"],
                        "Replies": 0,
                        "Likes": item["snippet"]["topLevelComment"]["snippet"]["likeCount"]}
                all_data.append(data)
            else:
                data = {"Thread": itemCount,
                        "Comment": item["snippet"]["topLevelComment"]["snippet"]["textOriginal"],
                        "Replies": 1,
                        "Likes": item["snippet"]["topLevelComment"]["snippet"]["likeCount"]}
                all_data.append(data)

                replyCount = 0
                for reply in item["replies"]["comments"]:
                    data = {"Thread": f"{itemCount}.{replyCount}",
                            "Comment": reply["snippet"]["textOriginal"],
                            "Replies": 2,
                            "Likes":reply["snippet"]["likeCount"]}
                    all_data.append(data)
                    replyCount += 1
            itemCount += 1

        next_page_token = response.get('nextPageToken')
    return all_data

In [7]:
#zhoTX0RRXPQ over 1000
#U2v76H_B1rs over 100
#JPHS10dt_CY 47
yt_data = YTcomments("U2v76H_B1rs")
print(pd.DataFrame(yt_data))

    Thread                                            Comment  Replies  Likes
0        0  I agree with every word you have said and I've...        1    100
1      0.0  Thanks for sharing Ranit! 👍 it’s good to also ...        2     17
2      0.1  @@Thuvu5 Yeah sure..... In Udemy, I particular...        2     55
3      0.2  @@ranitchatterjee5552 Woah this is so amazing,...        2     18
4      0.3  Could you guys provide links ? Sorry being laz...        2      0
..     ...                                                ...      ...    ...
320  211.1  @@Thuvu5 thanks in advance. Keep up the great ...        2      0
321    212  Hey there, thank you for watching! 👋Have you t...        1     20
322  212.0  Don't worry that's just a typo. You dont need ...        2      4
323  212.1  @@chhandomoydutta6397 Aw thank you!! And congr...        2      2
324  212.2  Hi Thu Vu, the typo is no big deal. I am almos...        2      1

[325 rows x 4 columns]


## Debuging section

In [8]:
request = youtube.commentThreads().list(part="replies, snippet", videoId="U2v76H_B1rs", maxResults="100", textFormat="html")
response = request.execute()
JSON(response)

<IPython.core.display.JSON object>

In [9]:
next_page_token = response.get('nextPageToken')
print(next_page_token)
request = youtube.commentThreads().list(part="id, replies, snippet", videoId="U2v76H_B1rs", maxResults="100", pageToken=next_page_token)
response = request.execute()
#JSON(response)

Z2V0X25ld2VzdF9maXJzdC0tQ2dnSWdBUVZGN2ZST0JJRkNJa2dHQUFTQlFpb0lCZ0FFZ1VJaUNBWUFCSUZDSWNnR0FBU0JRaWRJQmdCR0FBaURRb0xDT3kwbVpNR0VKRHI5VGM=


# Reddit

In [10]:
import requests
import json
from IPython.display import JSON

In [11]:
with open("/kaggle/input/credential/Reddit credential.txt", "r") as f:
    file = f.read()
file = file.split()

In [12]:
Client_ID = file[0]
Secret_key = file[1]
User_name = file[2]
Password = file[3]

In [13]:
auth = requests.auth.HTTPBasicAuth(Client_ID, Secret_key)

data = {
    "grant_type": "password",
    "username": User_name,
    "password": Password
}

headers = {"User-Agent": "MyAPI/0.0.1"}

res = requests.post("https://www.reddit.com/api/v1/access_token", auth=auth, data=data, headers=headers)

TOKEN = res.json()["access_token"]

headers["Authorization"] = f"bearer {TOKEN}"
#headers

In [14]:
# Subreddit posts
subreddit="StockMarket"
get_method="hot"
params={"limit":100,
        "threaded":True,
        "sort":"new"
       }
request= requests.get(f"https://oauth.reddit.com/r/{subreddit}/{get_method}", headers=headers, params=params)
request= request.json()
JSON(request)

<IPython.core.display.JSON object>

In [15]:
pd.json_normalize(request["data"]["children"],
                  #record_path=[""],
                  #meta=[""],
                  max_level=1)[["data.name", "data.link_flair_css_class", "data.upvote_ratio", "data.total_awards_received", "data.num_comments", "data.score", "data.title"]]

,data.name,data.link_flair_css_class,data.upvote_ratio,data.total_awards_received,data.num_comments,data.score,data.title
0,t3_1bsyq77,discussion,0.86,0,110,18,Rate My Portfolio - r/StockMarket Quarterly Th...
1,t3_1d0xc1j,discussion,0.67,0,11,1,Daily General Discussion and Advice Thread - M...
2,t3_1d15bpk,discussion,0.75,0,109,70,Are You Proud of the Companies You Invest In?
3,t3_1d1dxex,discussion,0.65,0,12,8,About 401k investment break up
4,t3_1d1gctt,flair-fa,0.64,0,2,3,JBLU is going to fly higher this summer
...,...,...,...,...,...,...,...
97,t3_1cu8c4x,discussion,0.42,0,97,0,When is the crash?
98,t3_1ct8jwk,discussion,0.85,0,6,13,Daily General Discussion and Advice Thread - M...
99,t3_1ctxi6k,discussion,0.22,0,2,0,Does anyone know why trading balance is low th...
100,t3_1ctcs5v,discussion,0.56,0,14,3,These are the stocks on my watchlist (5/16)


In [16]:
all_data = []
for children in request["data"]["children"]:
    data={"name": children["data"]["name"],
          "css_class": children["data"]["link_flair_css_class"],
          "upvote_ratio": children["data"]["upvote_ratio"],
          #"post_hint": children["data"]["post_hint"],
          "awards": children["data"]["total_awards_received"],
          "num_comments": children["data"]["num_comments"],
          "score": children["data"]["score"],
          "title": children["data"]["title"]}
    all_data.append(data)
df = pd.DataFrame(all_data)
df

,name,css_class,upvote_ratio,awards,num_comments,score,title
0,t3_1bsyq77,discussion,0.86,0,110,18,Rate My Portfolio - r/StockMarket Quarterly Th...
1,t3_1d0xc1j,discussion,0.67,0,11,1,Daily General Discussion and Advice Thread - M...
2,t3_1d15bpk,discussion,0.75,0,109,70,Are You Proud of the Companies You Invest In?
3,t3_1d1dxex,discussion,0.65,0,12,8,About 401k investment break up
4,t3_1d1gctt,flair-fa,0.64,0,2,3,JBLU is going to fly higher this summer
...,...,...,...,...,...,...,...
97,t3_1cu8c4x,discussion,0.42,0,97,0,When is the crash?
98,t3_1ct8jwk,discussion,0.85,0,6,13,Daily General Discussion and Advice Thread - M...
99,t3_1ctxi6k,discussion,0.22,0,2,0,Does anyone know why trading balance is low th...
100,t3_1ctcs5v,discussion,0.56,0,14,3,These are the stocks on my watchlist (5/16)


In [17]:
df.loc[df["num_comments"].idxmax()]

name                                                   t3_1d02ppc
css_class                                                    news
upvote_ratio                                                 0.91
awards                                                          0
num_comments                                                  507
score                                                        3251
title           GameStop surges after fetching $933 million fr...
Name: 9, dtype: object

In [18]:
#Post comments
postId= df.loc[df["num_comments"].idxmax()]["name"][3:]#all_data[4]["name"][3:]
params={#"limit":10000,
        #"threaded":True,
        #"sort":"new"
       }
request = requests.get(f"https://oauth.reddit.com/comments/{postId}", headers=headers)
request = request.json()
JSON(request)

<IPython.core.display.JSON object>

In [19]:
#json.loads(request.text)
#pd.read_json(request)
#pd.DataFrame.from_dict(request, orient='index')
pd.json_normalize(request,
                  record_path=["data", "children"],
                  max_level=1)[["data.replies", "data.body"]]#.loc[[3]]

,data.replies,data.body
0,NaN,NaN
1,"{'kind': 'Listing', 'data': {'after': None, 'd...",We have the opportunity to do the funniest thing
2,"{'kind': 'Listing', 'data': {'after': None, 'd...","Wow, they did it. Cant wait to see what they w..."
3,"{'kind': 'Listing', 'data': {'after': None, 'd...",1. They converted into Holding Company year ag...
4,"{'kind': 'Listing', 'data': {'after': None, 'd...",15% equity offering for almost $1 Billion. Now...
5,"{'kind': 'Listing', 'data': {'after': None, 'd...",Once you figure out why only buying digital ga...
6,,Good for game stonk
7,"{'kind': 'Listing', 'data': {'after': None, 'd...","\nWith 2 billion cash, 1 billion stock, no deb..."
8,,"It is better to have $1.2B on hand and wait, m..."
9,"{'kind': 'Listing', 'data': {'after': None, 'd...",Shorts r Fukd. \n\nGameStop has 2 BILLION fuck...


## Debugging session

In [20]:
params={"limit":100,
        "threaded":True,
        "sort":"new"
       }
request = requests.get('https://oauth.reddit.com/r/StockMarket/comments', headers=headers, params=params)
JSON(request.json())

<IPython.core.display.JSON object>

In [21]:
params={"limit":100,
        #"threaded":True,
        #"sort":"new"
       }
request = requests.get('https://oauth.reddit.com/morechildren', headers=headers, params=params)
JSON(request.json())

<IPython.core.display.JSON object>

In [22]:
#Post comments
linkId=all_data[5]
request = requests.get(f"https://oauth.reddit.com/comments/{linkId}", headers=headers)
JSON(request.json())

<IPython.core.display.JSON object>

# Twitter

# import requests
import os
import json

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("BEARER_TOKEN")

search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    json_response = connect_to_endpoint(search_url, query_params)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()